In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer

/Users/Roman/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Подготовка данных (urls)

In [5]:
urls_train_df = pd.read_csv('data/url_domain_train.csv', header=None, delimiter='\t')
urls_train_df.columns = ['id', 'url', 'count']
urls_train_df = urls_train_df[['id', 'url']]

IOError: File data/url_domain_train.csv does not exist

In [3]:
urls_train_df.head()

NameError: name 'urls_train_df' is not defined

In [ ]:
urls_train_df = pd.DataFrame(urls_train_df.groupby('id')['url'].apply(lambda x: x.tolist()))
urls_train_df['id'] = urls_train_df.index
urls_train_df.index = range(len(urls_train_df))
urls_train_df.columns = ['urls', 'id']

In [ ]:
urls_train_df.head()

In [ ]:
age_train_df = pd.read_csv('age_profile_train', header=None, delimiter='\t')
age_train_df.columns = ['id', 'age']

In [ ]:
age_train_df.head()

In [ ]:
train_df = urls_train_df.merge(age_train_df, on='id', how='left')

In [ ]:
train_df.head()

## Снижение размерности

In [ ]:
topk = 30000
X, y = train_df.urls.values[:topk], train_df.age.values[:topk]

In [ ]:
X = map(lambda x: ' '.join(x), X)
hw = HashingVectorizer(n_features=1000).fit(X)
X = hw.transform(X).todense()

## Обучение модели

In [ ]:
reg = LinearRegression()
- cross_val_score(reg, X, y, scoring='mean_squared_error')

## Отправка Решения

In [ ]:
reg = LinearRegression()
reg.fit(X, y)

In [ ]:
urls_test_df = pd.read_csv('url_domain_test', header=None, delimiter='\t')
urls_test_df.columns = ['id', 'url', 'count']
urls_test_df = urls_test_df[['id', 'url']]

In [ ]:
urls_test_df = pd.DataFrame(urls_test_df.groupby('id')['url'].apply(lambda x: x.tolist()))
urls_test_df['id'] = urls_test_df.index
urls_test_df.index = range(len(urls_test_df))
urls_test_df.columns = ['urls', 'id']

In [ ]:
urls_test_df.head()

In [ ]:
X = urls_test_df.urls.values
X = map(lambda x: ' '.join(x), X)
X = hw.transform(X).todense()

In [ ]:
y_pred = reg.predict(X)

In [ ]:
y_pred

In [ ]:
urls_test_df['age'] = y_pred

In [ ]:
urls_test_df = urls_test_df[['id', 'age']]
urls_test_df.columns = ['Id', 'age']

In [ ]:
urls_test_df.head()

In [ ]:
random_sol = pd.read_csv('random_solution.csv')
miss_idx = set(random_sol.Id.values) - set(urls_test_df.Id.values)
miss_df = pd.DataFrame(zip(list(miss_idx), np.ones(len(miss_idx))))
miss_df.columns = ['Id', 'age']

In [ ]:
urls_test_df = urls_test_df.append(miss_df, ignore_index=True)

In [ ]:
urls_test_df.to_csv('solution.csv', index=False)

In [ ]:
!wc -l solution.csv